In [1]:
import pandas as pd
import polars as pl
import json
import time
from sqlalchemy import create_engine

In [ ]:
def read_psc(file, column_to_normalize='data'):
    # Read the JSON file
    psc_df = pd.read_json(file, lines=True)
    print(psc_df)

    # Flatten the nested JSON within the specified column
    data_flattened = pd.json_normalize(psc_df[column_to_normalize])

    # Drop the original nested column and join with the flattened data
    psc_df = psc_df.drop(column_to_normalize, axis=1).join(data_flattened)

    return psc_df

read_psc("data/persons-with-significant-control-snapshot-2023-08-05.txt").to_csv("data/persons-with-significant-control-snapshot-2023-08-05.csv)

In [2]:
psc_df = pl.read_csv("data/persons-with-significant-control-snapshot-2023-08-05.csv")

In [3]:
comp_df = pl.read_csv("data/BasicCompanyDataAsOneFile-2023-08-01.csv")

In [4]:
comp_df = comp_df.rename({" CompanyNumber": "company_number"})

In [5]:
comp_df

CompanyName,company_number,RegAddress.CareOf,RegAddress.POBox,RegAddress.AddressLine1,RegAddress.AddressLine2,RegAddress.PostTown,RegAddress.County,RegAddress.Country,RegAddress.PostCode,CompanyCategory,CompanyStatus,CountryOfOrigin,DissolutionDate,IncorporationDate,Accounts.AccountRefDay,Accounts.AccountRefMonth,Accounts.NextDueDate,Accounts.LastMadeUpDate,Accounts.AccountCategory,Returns.NextDueDate,Returns.LastMadeUpDate,Mortgages.NumMortCharges,Mortgages.NumMortOutstanding,Mortgages.NumMortPartSatisfied,Mortgages.NumMortSatisfied,SICCode.SicText_1,SICCode.SicText_2,SICCode.SicText_3,SICCode.SicText_4,LimitedPartnerships.NumGenPartners,LimitedPartnerships.NumLimPartners,URI,PreviousName_1.CONDATE,PreviousName_1.CompanyName,PreviousName_2.CONDATE,PreviousName_2.CompanyName,PreviousName_3.CONDATE,PreviousName_3.CompanyName,PreviousName_4.CONDATE,PreviousName_4.CompanyName,PreviousName_5.CONDATE,PreviousName_5.CompanyName,PreviousName_6.CONDATE,PreviousName_6.CompanyName,PreviousName_7.CONDATE,PreviousName_7.CompanyName,PreviousName_8.CONDATE,PreviousName_8.CompanyName,PreviousName_9.CONDATE,PreviousName_9.CompanyName,PreviousName_10.CONDATE,PreviousName_10.CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""! HEAL UR TECH…","""14413082""","""""","""""","""5 BRIDGE STREE…","""""","""GUILDFORD""","""""","""ENGLAND""","""GU1 4RY""","""PRI/LTD BY GUA…","""Active""","""United Kingdom…","""""","""12/10/2022""","""31""","""10""","""12/07/2024""","""""","""NO ACCOUNTS FI…","""09/11/2023""","""""",0,0,0,0,"""33140 - Repair…","""47421 - Retail…","""""","""""",0,0,"""http://busines…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""25/10/2023""",""""""
"""! LTD""","""08209948""","""""","""""","""9 PRINCES SQUA…","""""","""HARROGATE""","""""","""ENGLAND""","""HG1 1ND""","""Private Limite…","""Active""","""United Kingdom…","""""","""11/09/2012""","""30""","""9""","""30/06/2024""","""30/09/2022""","""DORMANT""","""09/10/2016""","""11/09/2015""",0,0,0,0,"""99999 - Dorman…","""""","""""","""""",0,0,"""http://busines…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""25/09/2023""","""11/09/2022"""
"""!? LTD""","""11399177""","""""","""""","""THE STUDIO HAT…","""HATHERLOW""","""ROMILEY""","""""","""UNITED KINGDOM…","""SK6 3DY""","""Private Limite…","""Active""","""United Kingdom…","""""","""05/06/2018""","""30""","""6""","""31/03/2024""","""30/06/2022""","""TOTAL EXEMPTIO…","""03/07/2019""","""""",0,0,0,0,"""47710 - Retail…","""""","""""","""""",0,0,"""http://busines…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""19/06/2024""","""05/06/2023"""
"""!BIG IMPACT GR…","""11743365""","""""","""""","""372 OLD STREET…","""335 ROSDEN HOU…","""LONDON""","""""","""UNITED KINGDOM…","""EC1V 9LT""","""Private Limite…","""Active""","""United Kingdom…","""""","""28/12/2018""","""31""","""12""","""30/09/2023""","""31/12/2021""","""DORMANT""","""25/01/2020""","""""",0,0,0,0,"""18129 - Printi…","""59112 - Video …","""63120 - Web po…","""74201 - Portra…",0,0,"""http://busines…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""10/01/2024""","""27/12/2022"""
"""!GOBERUB LTD""","""13404790""","""""","""""","""13 SOUTHMILL R…","""""","""BISHOP'S STORT…","""HERTFORDSHIRE""","""ENGLAND""","""CM23 3FR""","""Private Limite…","""Active""","""United Kingdom…","""""","""17/05/2021""","""30""","""9""","""30/06/2024""","""30/09/2022""","""MICRO ENTITY""","""14/06/2022""","""""",0,0

In [10]:
def create_sqlite_db_from_df(psc_df, companies_df, db_path):
    connection_string = f"sqlite:///{db_path}"
    
    # Replace all periods in column names with underscores
    psc_df = psc_df.rename({'date_of_birth.month': 'date_of_birth_month',
                            'date_of_birth.year': 'date_of_birth_year',
                            'address.address_line_1': 'address_line_1',
                            'address.postal_code': 'postal_code'})
    
    companies_df = companies_df.rename({"CompanyName": "company_name"})
    
    # Create Persons table
    start_time = time.time()
    print("Creating 'individuals' table...")
    persons_df = psc_df.select(['name', 'date_of_birth_month', 'date_of_birth_year']).unique()
    persons_df.write_database("individuals", connection_string, if_exists='replace')
    print("'individuals' table created successfully in", time.time() - start_time, "seconds!")

    # Create Companies table
    start_time = time.time()
    print("Creating 'companies' table...")
    companies_df = companies_df.rename({
        "RegAddress.AddressLine1": "address_line_1",
        "RegAddress.PostCode": "postal_code"
    }).select(["company_number", "company_name", "address_line_1", "postal_code"])
    companies_df.write_database("companies", connection_string, if_exists='replace')
    print("'companies' table created successfully in", time.time() - start_time, "seconds!")

    # Create Addresses table
    start_time = time.time()
    print("Creating 'addresses' table...")
    addresses_df = psc_df.select(['address_line_1', 'postal_code']).unique()
    addresses_df.write_database("addresses", connection_string, if_exists='replace')
    print("'addresses' table created successfully in", time.time() - start_time, "seconds!")

    # Create PersonCompanyLink table
    start_time = time.time()
    print("Creating 'PersonCompanyLink' table...")
    person_company_df = psc_df.select(["name", "date_of_birth_month", "date_of_birth_year", "company_number"])
    person_company_df.write_database("PersonCompanyLink", connection_string, if_exists='replace')
    print("'PersonCompanyLink' table created successfully in", time.time() - start_time, "seconds!")

    # Create PersonAddressLink table
    start_time = time.time()
    print("Creating 'PersonAddressLink' table...")
    person_address_df = psc_df.select(["name", "date_of_birth_month", "date_of_birth_year", 'address_line_1', 'postal_code'])
    person_address_df.write_database("PersonAddressLink", connection_string, if_exists='replace')
    print("'PersonAddressLink' table created successfully in", time.time() - start_time, "seconds!")

    print("Database creation process completed!")


create_sqlite_db_from_df(psc_df, comp_df, "companies_house.db")


Creating 'individuals' table...
'individuals' table created successfully in 42.497090578079224 seconds!
Creating 'companies' table...
'companies' table created successfully in 34.162612199783325 seconds!
Creating 'addresses' table...
'addresses' table created successfully in 11.942702293395996 seconds!
Creating 'PersonCompanyLink' table...
'PersonCompanyLink' table created successfully in 73.65449738502502 seconds!
Creating 'PersonAddressLink' table...
'PersonAddressLink' table created successfully in 90.02910661697388 seconds!
Database creation process completed!


In [11]:
def get_columns_of_all_tables(db_path):
    engine = create_engine(f"sqlite:///{db_path}")
    conn = engine.connect()

    # Get list of all tables in the database
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

    table_columns = {}

    for table in tables:
        table_name = table[0]
        columns = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
        column_names = [column[1] for column in columns]
        table_columns[table_name] = column_names

    conn.close()
    return table_columns

db_path = "companies_house.db"
print(get_columns_of_all_tables(db_path))

{'individuals': ['name', 'date_of_birth_month', 'date_of_birth_year'], 'companies': ['company_number', 'company_name', 'address_line_1', 'postal_code'], 'addresses': ['address_line_1', 'postal_code'], 'PersonCompanyLink': ['name', 'date_of_birth_month', 'date_of_birth_year', 'company_number'], 'PersonAddressLink': ['name', 'date_of_birth_month', 'date_of_birth_year', 'address_line_1', 'postal_code']}


In [12]:
def add_indices_to_db(db_path):
    engine = create_engine(f"sqlite:///{db_path}")

    # individuals table indices
    engine.execute("CREATE INDEX idx_individual ON individuals(name, date_of_birth_month, date_of_birth_year)")

    # companies table indices
    engine.execute("CREATE INDEX idx_company_number ON companies(company_number)")

    # addresses table indices
    engine.execute("CREATE INDEX idx_address ON addresses(address_line_1, postal_code)")

    # PersonCompanyLink table indices
    engine.execute("CREATE INDEX idx_person_on_link ON PersonCompanyLink(name, date_of_birth_month, date_of_birth_year)")
    engine.execute("CREATE INDEX idx_company_on_link ON PersonCompanyLink(company_number)")

    # PersonAddressLink table indices
    engine.execute("CREATE INDEX idx_person_on_address_link ON PersonAddressLink(name, date_of_birth_month, date_of_birth_year)")
    engine.execute("CREATE INDEX idx_address_on_person_link ON PersonAddressLink(address_line_1, postal_code)")

    print("Indices added successfully!")
    
def add_fts_to_db(db_path):
    engine = create_engine(f"sqlite:///{db_path}")
    
    # Create the FTS table for company names
    engine.execute("""
    CREATE VIRTUAL TABLE IF NOT EXISTS companies_name_search 
    USING fts5(company_name, company_number UNINDEXED);
    """)
    
    # Insert company names and numbers into the FTS table
    engine.execute("""
    INSERT INTO companies_name_search(company_name, company_number) 
    SELECT company_name, company_number FROM companies;
    """)
    
    print("FTS created successfully!")

def optimize_db(db_path):
    engine = create_engine(f"sqlite:///{db_path}")

    # Apply the configurations
    engine.execute("PRAGMA journal_mode = delete;")
    engine.execute("PRAGMA page_size = 1024;")

    # Optimize the FTS table
    engine.execute("INSERT INTO companies_name_search(companies_name_search) VALUES ('optimize');")

    # Vacuum the database
    engine.execute("VACUUM;")

    print("Database optimized successfully!")

add_indices_to_db("companies_house.db")
add_fts_to_db("companies_house.db")
optimize_db("companies_house.db")

Indices added successfully!
FTS created successfully!
Database optimized successfully!
